<a href="https://colab.research.google.com/github/esdrasantos/Aulas-de-programacao/blob/master/cstfm_mod_9_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import datetime

# Buscando dados de tabelas .csv hospedadas no google drive⚡

In [ ]:
dir = '/content/drive/MyDrive/faturas_nu_csv'

arquivos = [arquivo for arquivo in os.listdir(dir)]

padrao = f"_(.*?)-"

arquivosord = sorted(arquivos, key=lambda x: int(re.findall(padrao, x)[0]))
arquivosord = sorted(arquivosord, key=lambda x: int(x.split('-')[1]))
arquivosord

In [ ]:
dados = pd.DataFrame()

for arquivo in arquivosord:
  tabela = pd.read_csv(f'{dir}/{arquivo}')
  tabela['Arquivo Origem'] = arquivo
  dados = pd.concat([dados,tabela])

dados.reset_index(drop=True,inplace=True)

In [ ]:
dados['Fech. Fatura'] = dados['Arquivo Origem']\
.str.rsplit('_', n=1).str[1]\
.str.split('.').str[0]

# Corrigindo as datas das observações e preparando dados ⛴

In [ ]:
dia_mes = dados['Data'].str.split(' ')
dados['Dia'] = dia_mes.str[0].str.strip()
dados['Mes'] = dia_mes.str[1].str.strip()
dados['Ano'] = dados['Fech. Fatura'].str\
              .split('-').str[0]\
              .str.strip()
meses = {
    "JAN": 1,
    "FEV": 2,
    "MAR": 3,
    "ABR": 4,
    "MAI": 5,
    "JUN": 6,
    "JUL": 7,
    "AGO": 8,
    "SET": 9,
    "OUT": 10,
    "NOV": 11,
    "DEZ": 12
}

dados['N Mes'] = dados['Mes'].map(meses)

dados['Data_Compra_Aux'] = dados.Ano + "-"\
                           + dados['N Mes'].astype(str) + "-"\
                           + dados.Dia

In [ ]:
dados.drop(columns=['Dia','Mes','Ano','N Mes'],inplace=True)

In [ ]:
dtdiff = pd.to_datetime(dados['Fech. Fatura'])-pd.to_datetime(dados['Data_Compra_Aux'])
dados['Dias_Compra'] = dtdiff.dt.days

In [ ]:
dados.drop(columns=["Data"], inplace=True)

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
dados['Data'] = pd.to_datetime(dados['Data_Compra_Aux'])
filtro = dados['Dias_Compra'] < 0
dados.loc[filtro, 'Data'] = dados.loc[filtro, 'Data'] - pd.DateOffset(years=1)

In [ ]:
dados.drop(columns=['Data_Compra_Aux', 'Dias_Compra'],inplace=True)

In [ ]:
dados['Mes Compra'] = dados['Data'].dt.month
dados['Semana Compra'] = dados['Data'].dt.isocalendar().week

In [ ]:
def semana_do_mes(data):
    return data.day // 7 + 1

semana_do_mes(pd.to_datetime('2025-12-31'))

In [ ]:
datas = dados[['Data']].copy()
print(datas.head())


In [ ]:
datas['Semana do Mes'] = datas.apply(lambda x: semana_do_mes(pd.to_datetime(x['Data'])), axis=1)

print(datas[-15:])

In [ ]:
dados['Semana do Mes'] = datas['Semana do Mes']
dados['Ano'] = datas['Data'].dt.year
dados['Mes'] = datas['Data'].dt.month

In [ ]:
dados.drop(columns=['Arquivo Origem', 'Fech. Fatura', 'Compra'],inplace=True)
print(dados.head())

# Definindo dimensões para ANOVA 1 VIA

In [ ]:
df = dados.copy()
df['Valor'] = df['Valor'].astype(str)
df['Valor'] = df['Valor'].str.replace('R$', '')
df['Valor'] = df['Valor'].str.replace(',', '.')
df['Valor'] = df['Valor'].str.replace(r'\.(?=.*\.)', '', regex=True)

In [ ]:
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')


In [ ]:
df['Valor'].agg(['min','max','mean'])

In [ ]:
print(df.tail())

In [ ]:
quintas = df[df['Data'].dt.weekday == 3]

In [ ]:
# media_quintas = quintas.groupby(['Ano', 'Mes', 'Semana do Mes'])['Valor'].mean().reset_index()

quintas.drop(columns=['Categoria', 'Data', 'Semana Compra', 'Mes Compra'], inplace=True)

In [ ]:
print(quintas.tail())

In [ ]:
media_quintas = quintas.copy()

media_quintas['Ano Mes'] = quintas['Ano'].astype(str) + '-' + quintas['Mes'].astype(str)

media_quintas.drop(columns=['Ano', 'Mes'], inplace=True)

# ANOVA DE 1 VIA

Repondamos a seguinte pergunta: \
As médias de gasto nas quintas-feiras de cada mês são estatisticamente iguais? \
Se forem diferentes, em quais meses essas médias são estatisticamente diferentes? \

Ho: μ1=μ2=μ3=...=μ12 \
Ha: ∃ μi, μj | μi != μj

ou, simplesmente \

Ho: μ1=μ2=μ3=...=μ12 \
Ha: Existe ao menos uma média diferente das demais.


In [ ]:
print(media_quintas.tail())  # Ultimos 5

In [ ]:
g_sum_quintas = media_quintas.groupby(['Ano Mes', 'Semana do Mes'])['Valor'].sum()
print(g_sum_quintas.head())

In [ ]:
from scipy.stats import f_oneway

In [ ]:
# Tratamento de NAN

g_sum_quintas.dropna(inplace=True)

In [ ]:
c2412 = g_sum_quintas['2024-12']
c2501 = g_sum_quintas['2025-1']
c2502 = g_sum_quintas['2025-2']
c2503 = g_sum_quintas['2025-3']
c2504 = g_sum_quintas['2025-4']
c2505 = g_sum_quintas['2025-5']
c2506 = g_sum_quintas['2025-6']
c2507 = g_sum_quintas['2025-7']
c2508 = g_sum_quintas['2025-8']
c2509 = g_sum_quintas['2025-9']
c2510 = g_sum_quintas['2025-10']

print(c2510, c2509)

In [ ]:
f_stat, p_value = f_oneway(c2412, c2501, c2502, c2503, c2504, c2505, c2506, c2507, c2508, c2509, c2510)

print("F-statistic:", f_stat)
print("p-value:", p_value)

In [ ]:
# Decisão:

if p_value < 0.05:
    print("As médias de gasto nas quintas-feiras de cada mês são estatisticamente diferentes.")
else:
    print("As médias de gasto nas quintas-feiras de cada mês são estatisticamente iguais.")

# TUKEY DATAFRAME PARA TESTE TUKEY

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df_tukey = pd.DataFrame({
    'Valores': g_sum_quintas,
    'Ano Mes': g_sum_quintas.index.get_level_values(0),
    'Semana do Mes': g_sum_quintas.index.get_level_values(1)
})

df_tukey.reset_index(drop=True, inplace=True)
df_tukey.head()

In [ ]:
tukey = pairwise_tukeyhsd(df_tukey['Valores'], df_tukey['Ano Mes'], alpha=0.05)

print(tukey)

# p-adj é p-valor ajustado

In [ ]:
# Decisão
rejeitou = "Sim" if tukey.reject.all() else "Não"

print("Algum valor rejeitou Ho?", rejeitou)

# Printar quem reijeitou a hipotese nula no tukey:

for i in range(len(tukey.reject)):
    if tukey.reject[i]:
        print('Rejeitou a hipótese nula: ', i)

# Em exemplo de pivot dos dados


In [ ]:
c202412 = media_quintas[media_quintas['Ano Mes'] == '2024-12']['Valor']
c202501 = media_quintas[media_quintas['Ano Mes'] == '2025-1']['Valor']
c202502 = media_quintas[media_quintas['Ano Mes'] == '2025-2']['Valor']
c202503 = media_quintas[media_quintas['Ano Mes'] == '2025-3']['Valor']
c202504 = media_quintas[media_quintas['Ano Mes'] == '2025-4']['Valor']
c202505 = media_quintas[media_quintas['Ano Mes'] == '2025-5']['Valor']
c202506 = media_quintas[media_quintas['Ano Mes'] == '2025-6']['Valor']
c202507 = media_quintas[media_quintas['Ano Mes'] == '2025-7']['Valor']
c202508 = media_quintas[media_quintas['Ano Mes'] == '2025-8']['Valor']
c202509 = media_quintas[media_quintas['Ano Mes'] == '2025-9']['Valor']
c202510 = media_quintas[media_quintas['Ano Mes'] == '2025-10']['Valor']

print(c202412.head())

In [ ]:
f_stat, p_value = f_oneway(c202412, c202501, c202502, c202503, c202504, c202505, c202506, c202507, c202508, c202509, c202510)

print("F-statistic:", f_stat)
print("p-value:", p_value)

In [ ]:
media_quintas.reset_index(drop=True, inplace=True)
print(media_quintas.tail())

In [ ]:
# Só para ficar igual ao exercício do curso - com o fator nas colunas e o valor nas linhas.

cumcount = media_quintas.groupby('Ano Mes').cumcount()

pv_quintas = media_quintas.set_index(cumcount).pivot(columns='Ano Mes', values='Valor')

print("Head:\n", pv_quintas.head())

print("Tail:\n", pv_quintas.tail())

In [ ]:
# Tudo isso para porque f_oneaway anova não suporta NaN
# E também tive de pivotar os dados para ficar similar à
# prática do curso...
# O arquivo <> tem um exemplo mais simples

dfs = {
    col: pv_quintas[col].dropna().to_list()
    for col in pv_quintas.columns
}

In [ ]:
[df for df in dfs.values()][:2]

In [ ]:
f_stat, p_value = f_oneway(
    *[
        df
        for df in dfs.values()
    ]
)
print("F-statistic:", f_stat)
print("p-value:", p_value)